In [1]:
import os
os.chdir('../')

import src.scrape_apk as scraper
import src.create_matrices as cm
import src.model as model
import json
import src.Malware_Types as mt
from collections import defaultdict
import pandas as pd

from random import sample


In [2]:
params = json.load(open('config/data-params.json'))
categories = params['categories'] 
scrape_data = params['scrape_data']
numer_of_apps_per_cat = params['number_of_apps_per_cat']
outpath = params['outpath']
malware_categories = ["Andup","BankBot","Boxer","DroidKungFu","FakeAngry","FakeInst","FakeAV","FakeDoc","Gorpo",\
                        "Fjcon","GingerMaster","Kemoge","Koler","Ksapp","Kyview","Lotoor","Minimob","MobileTX",\
                        "Mtk","Nandrobox","Obad","Roop","RuMMS","SimpleLocker","SmsZombie","SpyBubble","Stealer",\
                        "Steek","Univert","Vidro","VikingHorde","Vmvol","Youmi"]
malware_path = params['malware_path']
num_malware_apps = params['num_malware_apps']
re_read = params['re_read']

In [3]:
categories.remove('tools')

In [4]:
all_apis = json.load(open('./data/apis.json'))

In [5]:
all_apps = json.load(open('./data/apps.json'))

In [6]:
def get_benign():
    category_data,apps,apis,test_info, test_apps = cm.get_metadata(categories, outpath,False)
    return category_data,apps,apis

In [7]:
category_data,apps,apis = get_benign()

beauty
business
dating
game
sports


In [8]:
len(apps)

382

In [9]:
len(apis)

952450

In [10]:
df = mt.getMalwareData()

In [11]:
api_counts = {}

In [12]:
def compute_api_counts(category_data,api_counts):
    for cat, apps in category_data.items():
        if cat in malware_categories:
            cat = mt.condensedFaimlyData(df, cat)['Malware_Type'].iloc[0]
            if 'Trojan' in cat:
                cat = 'Trojan'
        else:
            cat = 'benign'
#         print(cat)
        cat_apis = defaultdict(int)
        for app, meta in apps.items():
            for api, count in meta['app_apis'].items():
                cat_apis[api] += count
        api_counts[cat] = cat_apis
    return api_counts

In [13]:
api_counts = compute_api_counts(category_data, api_counts)

In [14]:
def get_malware_cat(cats):
    category_data,apps,apis, malware,test_info, test_apps, malware_test = cm.read_malware(cats, outpath,{},{},{},{}, {})
    return category_data,apps,apis

In [15]:
adware = ["Andup","Kyview","Youmi","Minimob"]
backdoor = ['DroidKungFu','FakeAngry','Fjcon','GingerMaster','Obad','Univert']
ransom = ['Koler', 'Roop', "SimpleLocker"]
trojan = ['BankBot','Boxer','FakeInst','FakeAV','FakeDoc','Gorpo','Kemoge','Ksapp','MobileTX','Mtk','Nandrobox','RuMMS','SmsZombie','SpyBubble','Stealer','Steek','Vidro','VikingHorde','Vmvol']

In [16]:
adware_data,adware_apps,adware_apis = get_malware_cat(adware)

Andup
Kyview
Youmi
Minimob


In [17]:
len(adware_apps)

164

In [18]:
len(adware_apis)

152969

In [19]:
api_counts = compute_api_counts(adware_data, api_counts)

In [20]:
adware_counts = pd.DataFrame(api_counts)
adware_counts = adware_counts.fillna(0)
adware_counts['diff'] = adware_counts['Adware'] - adware_counts['benign']
adware_counts['api_id'] = adware_counts.index.map(all_apis)

In [21]:
adware_counts.sort_values('diff', ascending = False).head(10)

,benign,Adware,diff,api_id
Lcom/google/android/gms/common/internal/safeparcel/b;->a()V,3187.0,9589.0,6402.0,44162.0
Lorg/lwjgl/BufferChecks;->checkFunctionAddress()V,0.0,5640.0,5640.0,NaN
Lorg/lwjgl/opengl/GLContext;->getCapabilities()Lorg/lwjgl/opengl/ContextCapabilities;,0.0,5344.0,5344.0,NaN
Lorg/lwjgl/MemoryUtil;->getAddress()J,0.0,4520.0,4520.0,NaN
Lorg/lwjgl/opengl/GLContext;->getFunctionAddress()J,0.0,4140.0,4140.0,NaN
Lcom/google/android/gms/common/internal/safeparcel/a;->a()I,1981.0,5353.0,3372.0,270.0
Lcom/badlogic/gdx/backends/android/surfaceview/GLLogWrapper;->arg()V,0.0,3152.0,3152.0,212667.0
Lorg/lwjgl/BufferChecks;->checkDirect()V,0.0,2904.0,2904.0,NaN
Landroid/content/Intent;->putExtra()Landroid/content/Intent;,7177.0,9102.0,1925.0,15.0
Landroid/content/Intent;->getStringExtra()Ljava/lang/String;,1874.0,3663.0,1789.0,1990.0


In [178]:
del api_counts['Adware']

In [22]:
backdoor_data,backdoor_apps,backdoor_apis = get_malware_cat(backdoor)

DroidKungFu
FakeAngry
Fjcon
GingerMaster
Obad
Univert


In [23]:
len(backdoor_apps)

130

In [24]:
len(backdoor_apis)

59006

In [25]:
api_counts = compute_api_counts(backdoor_data, api_counts)

In [26]:
api_counts.keys()

dict_keys(['benign', 'Adware', 'Backdoor'])

In [27]:
backdoor_counts = pd.DataFrame(api_counts)
backdoor_counts = backdoor_counts.fillna(0)
backdoor_counts['diff'] = backdoor_counts['Backdoor'] - backdoor_counts['benign']
backdoor_counts['api_id'] = backdoor_counts.index.map(all_apis)

In [28]:
backdoor_counts.sort_values('diff', ascending = False).head(10)

,benign,Adware,Backdoor,diff,api_id
Lnet/thesubservices/resserv/TheCheckService;->getBaseContext()Landroid/content/Context;,0.0,0.0,70.0,70.0,234072.0
Lnet/thesubservices/resserv/TheCheckService;->getSystemService()Ljava/lang/Object;,0.0,0.0,49.0,49.0,234070.0
Landroid/support/v4/widget/DrawerLayout;->checkDrawerViewGravity()Z,8.0,24.0,56.0,48.0,210693.0
Landroid/telephony/gsm/SmsMessage;->getOriginatingAddress()Ljava/lang/String;,0.0,0.0,42.0,42.0,197800.0
Landroid/support/v4/util/LongSparseArray;->binarySearch()I,5.0,55.0,35.0,30.0,191689.0
Landroid/support/v4/util/SparseArrayCompat;->binarySearch()I,5.0,55.0,35.0,30.0,191686.0
Landroid/support/v4/widget/DrawerLayout;->getDrawerViewGravity()I,5.0,15.0,35.0,30.0,210692.0
Lnet/thesubservices/resserv/SmsReceiver;->abortBroadcast2()V,0.0,0.0,21.0,21.0,234067.0
Landroid/telephony/gsm/SmsMessage;->getMessageBody()Ljava/lang/String;,0.0,0.0,21.0,21.0,197799.0
Landroid/support/v4/util/SparseArrayCompat;->idealIntArraySize()I,3.0,33.0,21.0,18.0,191684.0


In [183]:
del api_counts['Backdoor']

In [29]:
ransom_data,ransom_apps,ransom_apis = get_malware_cat(ransom)

Koler
Roop
SimpleLocker


In [30]:
len(ransom_apps)

110

In [31]:
len(ransom_apis)

16832

In [32]:
api_counts = compute_api_counts(ransom_data, api_counts)

In [33]:
api_counts.keys()

dict_keys(['benign', 'Adware', 'Backdoor', 'Ransom'])

In [34]:
ransom_counts = pd.DataFrame(api_counts)
ransom_counts = ransom_counts.fillna(0)
ransom_counts['diff'] = ransom_counts['Ransom'] - ransom_counts['benign']
ransom_counts['api_id'] = ransom_counts.index.map(all_apis)

In [35]:
ransom_counts.sort_values('diff', ascending = False).head(10)

,benign,Adware,Backdoor,Ransom,diff,api_id
Lorg/spongycastle/asn1/ASN1EncodableVector;->add()V,0.0,0.0,0.0,1091.0,1091.0,NaN
Lorg/spongycastle/asn1/ASN1Sequence;->getObjectAt()Lorg/spongycastle/asn1/ASN1Encodable;,0.0,0.0,0.0,677.0,677.0,NaN
Lch/boye/httpclientandroidlib/util/Args;->notNull()Ljava/lang/Object;,0.0,0.0,0.0,671.0,671.0,NaN
Lorg/spongycastle/asn1/ASN1ObjectIdentifier;->branch()Lorg/spongycastle/asn1/ASN1ObjectIdentifier;,0.0,0.0,0.0,445.0,445.0,NaN
Lorg/spongycastle/asn1/ASN1Sequence;->getInstance()Lorg/spongycastle/asn1/ASN1Sequence;,0.0,0.0,0.0,372.0,372.0,NaN
Lorg/spongycastle/asn1/ASN1Sequence;->size()I,0.0,0.0,0.0,368.0,368.0,NaN
Lorg/spongycastle/crypto/digests/RIPEMD320Digest;->RL()I,0.0,0.0,0.0,320.0,320.0,NaN
Lorg/spongycastle/crypto/digests/RIPEMD160Digest;->RL()I,0.0,0.0,0.0,320.0,320.0,NaN
Landroid/support/v4/app/s;->a()V,13.0,0.0,0.0,321.0,308.0,87806.0
Lcom/Rockstargames/b/h;->a()V,0.0,0.0,0.0,280.0,280.0,232381.0


In [190]:
del api_counts['Ransom']

In [36]:
trojan_data,trojan_apps,trojan_apis = get_malware_cat(trojan)

BankBot
Boxer
FakeInst
FakeAV
FakeDoc
Gorpo
Kemoge
Ksapp
MobileTX
Mtk
Nandrobox
RuMMS
SmsZombie
SpyBubble
Stealer
Steek
Vidro
VikingHorde
Vmvol


In [37]:
len(trojan_apps)

384

In [38]:
len(trojan_apis)

163415

In [39]:
api_counts = compute_api_counts(trojan_data, api_counts)

In [40]:
api_counts.keys()

dict_keys(['benign', 'Adware', 'Backdoor', 'Ransom', 'Trojan'])

In [41]:
trojan_counts = pd.DataFrame(api_counts)
trojan_counts = trojan_counts.fillna(0)
trojan_counts['diff'] = trojan_counts['Trojan'] - trojan_counts['benign']
trojan_counts['api_id'] = trojan_counts.index.map(all_apis)

In [42]:
trojan_counts.sort_values('diff', ascending = False).head(10)

,benign,Adware,Backdoor,Ransom,Trojan,diff,api_id
Lcom/google/gson/internal/bind/MiniGson$Builder;->factory()Lcom/google/gson/internal/bind/MiniGson$Builder;,0.0,0.0,0.0,0.0,297.0,297.0,231296.0
Lcom/google/gson/internal/bind/TypeAdapters;->newFactory()Lcom/google/gson/internal/bind/TypeAdapter$Factory;,0.0,0.0,0.0,0.0,189.0,189.0,231298.0
Lorg/apache/http/entity/mime/HttpMultipart;->writeBytes()V,0.0,0.0,0.0,0.0,171.0,171.0,44392.0
Lcom/mvlove/util/LogUtil;->debug()V,0.0,0.0,0.0,0.0,162.0,162.0,234393.0
Lcom/google/gson/internal/bind/JsonElementReader;->popStack()Ljava/lang/Object;,0.0,0.0,0.0,0.0,99.0,99.0,231322.0
Lcom/google/gson/stream/JsonReader;->quickPeek()Lcom/google/gson/stream/JsonToken;,0.0,0.0,0.0,0.0,90.0,90.0,44143.0
Lcom/google/gson/stream/JsonReader;->advance()Lcom/google/gson/stream/JsonToken;,0.0,0.0,0.0,0.0,81.0,81.0,44144.0
Lcom/google/gson/internal/bind/MiniGson;->getAdapter()Lcom/google/gson/internal/bind/TypeAdapter;,0.0,0.0,0.0,0.0,81.0,81.0,231303.0
Lcom/google/gson/internal/bind/JsonElementReader;->peek()Lcom/google/gson/stream/JsonToken;,0.0,0.0,0.0,0.0,81.0,81.0,231319.0
Lcom/google/gson/internal/bind/TypeAdapter;->write()V,0.0,0.0,0.0,0.0,72.0,72.0,231305.0


In [220]:
del api_counts['Trojan'] 

In [43]:
malware_counts = pd.DataFrame(api_counts)
malware_counts = malware_counts.fillna(0)
malware_counts['malware_counts_sum'] = malware_counts.Trojan + malware_counts.Backdoor + malware_counts.Ransom + malware_counts.Adware
malware_counts['diff'] = malware_counts.benign - malware_counts.malware_counts_sum

In [45]:
malware_counts[['benign','malware_counts_sum','diff']].sort_values('diff', ascending = False).head(10)

,benign,malware_counts_sum,diff
Ljava/lang/StringBuilder;->append()Ljava/lang/StringBuilder;,240047.0,94548.0,145499.0
Ljava/lang/StringBuilder;->toString()Ljava/lang/String;,76047.0,31810.0,44237.0
Ljava/lang/Integer;->valueOf()Ljava/lang/Integer;,37385.0,9108.0,28277.0
Ljava/lang/String;->equals()Z,38523.0,10542.0,27981.0
Lkotlin/jvm/internal/Intrinsics;->checkParameterIsNotNull()V,22498.0,0.0,22498.0
Ljava/lang/String;->valueOf()Ljava/lang/String;,22877.0,2474.0,20403.0
Ljava/util/Iterator;->next()Ljava/lang/Object;,25910.0,5524.0,20386.0
Ljava/util/Iterator;->hasNext()Z,25804.0,5495.0,20309.0
Ljava/lang/String;->length()I,23292.0,3704.0,19588.0
Landroid/util/Log;->a()I,27504.0,8401.0,19103.0


In [131]:
from pathlib import Path
import glob

In [132]:
pathlist = Path('data/APK').glob('**/**/*.smali')
smalli_lis = [i for i in pathlist]
print(len(smalli_lis))
smalli_lis = [i for i in pathlist if '$' not in str(i)]
print(len(smalli_lis))

KeyboardInterrupt: 

In [28]:
ma = 

In [32]:
for cat in ma:
    family=mt.condensedFaimlyData(df, cat)['Malware_Type'].iloc[0]
    if 'Trojan' in family:
        print(cat,family)

BankBot Trojan-Banker
Boxer Trojan-SMS
FakeInst Trojan-SMS
FakeAV Trojan
FakeDoc Trojan
Gorpo Trojan-Dropper
Kemoge Trojan-Dropper
Ksapp Trojan
MobileTX Trojan
Mtk Trojan
Nandrobox Trojan
RuMMS Trojan-SMS
SmsZombie Trojan-Spy
SpyBubble Trojan-SMS
Stealer Trojan-SMS
Steek Trojan-Clicker
Vidro Trojan-SMS
VikingHorde Trojan-Dropper
Vmvol Trojan-Spy


In [ ]:
category_data,apps,apis, malware,test_info, test_apps, malware_test = cm.read_malware(malware_cats, outpath,category_data,apps,apis,test_info, test_apps)

In [6]:
def get_cat_info(cat,dic):
    cat_lens = []
    cat_sum_avg = []
    for app, meta in dic[cat].items():
        total_count = 0
        for api, app_counts in meta['app_apis'].items():
            cat_lens.append(len(api))
            total_count += app_counts
        cat_sum_avg.append(total_count)
    print('Number of unique APIs called: ', np.mean(cat_lens))
    print('Average number of APIs called: ', np.mean(cat_sum_avg))

In [8]:
for cat in cats:
    print(cat)
    get_cat_info(cat,category_data)

beauty
Number of unique APIs called:  66.62952194638746
Average number of APIs called:  111191.06
business
Number of unique APIs called:  71.28186721749412
Average number of APIs called:  105133.18
dating
Number of unique APIs called:  66.07991253626204
Average number of APIs called:  165619.88
game
Number of unique APIs called:  66.54738747744929
Average number of APIs called:  89730.46
sports
Number of unique APIs called:  68.47899124307501
Average number of APIs called:  141453.0


In [2]:
malware = ["Andup","Kyview","Minimob","Youmi"]
category_data_malware,apps_malware,apis_malware, _ = cm.read_malware(malware, '../data',{},{},{})

Andup
Kyview
Minimob
Youmi


In [11]:
for cat in malware:
    print(cat)
    get_cat_info(cat,category_data_malware)

Andup
Number of unique APIs called:  50.11244411523505
Average number of APIs called:  17564.733333333334
Kyview
Number of unique APIs called:  54.99677211691352
Average number of APIs called:  36706.35
Minimob
Number of unique APIs called:  63.24201496995149
Average number of APIs called:  38617.5
Youmi
Number of unique APIs called:  53.98063804456381
Average number of APIs called:  31368.1


In [12]:
apis_malware.keys() - 

{'Ljava/lang/Object;-><init>()V': 0,
 'Lcom/atrace/complete/AppWallActivity;->access$000()Z': 1,
 'Lcom/atrace/complete/AppWallActivity;->access$002()Z': 2,
 'Lcom/atrace/complete/AppWallActivity;->access$100()Ljava/util/Map;': 3,
 'Ljava/lang/Integer;->valueOf()Ljava/lang/Integer;': 4,
 'Ljava/util/Map;->containsKey()Z': 5,
 'Landroid/os/Bundle;-><init>()V': 6,
 'Ljava/util/Map;->get()Ljava/lang/Object;': 7,
 'Landroid/os/Bundle;->putString()V': 8,
 'Lcom/atrace/complete/AppWallActivity;->access$200()I': 9,
 'Lcom/atrace/complete/AppWallActivity;->showDialog()Z': 10,
 'Lcom/atrace/complete/AppWallActivity;->access$300()I': 11,
 'Lcom/atrace/complete/AppWallActivity;->showDialog()V': 12,
 'Ljava/lang/StringBuilder;-><init>()V': 13,
 'Lcom/atrace/complete/utils/Tools;->getRealUrl()Ljava/lang/String;': 14,
 'Ljava/lang/StringBuilder;->append()Ljava/lang/StringBuilder;': 15,
 'Lcom/atrace/complete/AppWallActivity;->access$400()Ljava/lang/String;': 16,
 'Lcom/atrace/complete/AppWallActivit